In [ ]:
# Default spec, this cell will be overridden
state = {"spec": { "cloud": "aws" } }

In [ ]:
%run ../lib/all.py

In [ ]:
state = teststate.init_state(state)
state

In [ ]:
kops.populate_state()

In [ ]:
state

In [ ]:
kubetest = e2e.download_kubetest(teststate.get_state("spec.cluster.kubernetesVersion"))

In [ ]:
kops = kops.download_kops(teststate.get_state("spec.kopsVersion"))
kops.version()

In [ ]:
cluster_name = teststate.get_state("spec.cluster.name")

In [ ]:
c = kops.create_cluster(teststate.get_state("spec"))

In [ ]:
c = kops.cluster(cluster_name)

In [ ]:
c

In [ ]:
teststate.update_state({ "kops_spec": c.objects()})

In [ ]:
c.update()

In [ ]:
c.wait()

### Run tests

In [ ]:
notebooks.run("check-nodes")

In [ ]:
notebooks.run("check-storage-classes")

In [ ]:
#run_notebook("run-e2e", parameters={ "state": { "e2e": {"focus": "Secrets should be consumable via the ", "parallel": 25}}})
notebooks.run("run-e2e", parameters={ "state": { "e2e": {"focus": "\[Conformance\]", "skip": "\[Serial\]", "parallel": 25}}})

### Cleanup

In [ ]:
c.delete()

In [ ]:
teststate.finished()